In [65]:
import pandas as pd
from json import load
import numpy as np
from datetime import datetime
import pytz
from os import listdir
from os.path import getmtime, exists, isdir, isfile
from pathlib import Path

from openpyxl import load_workbook
from openpyxl.styles import Font, PatternFill
from urlextract import URLExtract
import re

#IP2024119
import shutil
from pathlib import Path



In [66]:
# how many rows demonstrate in executional cells 
rows_to_show = 1   # uses for debug with Jupiter's-cells-system


# !  IP20241118 naming like "flag_analyze_all" & "info_write_all_channels" is more understandable from glance

## Do you wish to convert all the Slack channels?:
analyze_all_flag = True   # False            #AG

## If not, insert name of Slack channel to convert:
exportname = '' # "general"

## Generate file with the information of all the Slack channels?:
write_all_channels_info = True
## Generate file with the information of all the Slack users?:
write_all_users_info = True


#working_directory = "..." #AG: not needed, enough with slackexport_folder_path and exporting_directory
#working_directory = 'E:\_RET_slack_export\RebeccaEverlene Slack export Apr 30 2021 - Oct 3 2024-2' 

## Insert path where the LOCAL copy of the GoogleDrive folder is:
#slackexport_folder_path = "/home/agds/Documents/RebeccaEverleneTrust/App/RebeccaEverlene_Slack_export" #AG
slackexport_folder_path = 'E:\_RET_slack_export\RebeccaEverlene Slack export Apr 30 2021 - Oct 3 2024-2short' #IP
#slackexport_folder_path = f"{working_directory}\{exportname}"


# ! IP20241118 if "exporting_directory" in fact means "directory-to-where-to-convert" 
#    - it shoild be renamed as "converted_directory".  !!  diminish any mess with naming of variables

#exporting_directory = "/home/agds/Downloads" #AG
exporting_directory = 'E:\_RET_slack_export\RebeccaEverlene Slack export Apr 30 2021 - Oct 3 2024-2short' #IP

exporting_directory = f"{exporting_directory}/JSONs_converted"
## IP20241119  Check that slackexport_folder_path exists:
if exists(exporting_directory)==True:
    exprt_folder_path = Path(exporting_directory)
    if exprt_folder_path.is_dir():
        shutil.rmtree(exprt_folder_path)
        
Path(f"{exporting_directory}").mkdir(parents=True, exist_ok=True) #IP20241119
        

         
#Path(f"{exporting_directory}/JSONs_converted").mkdir(parents=True, exist_ok=True) #AG

# IP20241118
#exporting_directory = f"{exporting_directory}/RET_converted"

#exporting_directory = f"{working_directory}\{'_converted'}"  

# !!!  IP20241118  above is not ok.: code need to check - if the 'RET_converted'  exist.
# if exist - delete current 'RET_converted' . 
# then create new fresh-&-empty 'RET_converted'


In [67]:
#AG: Added a routine that checks the existence/format of files/directories of the channel(s) requested by the user.

# !!! IP20241118 should check before - is current file a *.json or file extention is different(then skip this file)
def check_jsonFile_nameFormat(file_name):
    """ Checks the naming-format of a json file. 
    Returns True if the name of the file is of the type yyyy-mm-dd.json"""
    list_json_format = file_name.split(".json")[0].split("-")   ### COULD BE IMPROVED
    try:
        if len(list_json_format[0])==4 and len(list_json_format[1])==2 and len(list_json_format[2])==2:
            return True
    except:
        return False

def review_format_of_jsonFiles(list_names):
    """ Iterates over all the json files in a channel's directory, and separates the ones with the
    correct date-format (list_names_dates) from the rest (list_names_others).
    It uses the funtion check_jsonFile_nameFormat """
    list_names_dates = []
    list_names_others = []
    for i in range(len(list_names)):
        if check_jsonFile_nameFormat(list_names[i])==True:
            list_names_dates.append(list_names[i])
        else:
            list_names_others.append(list_names[i])
    return list_names_dates, list_names_others
    
## Analyze directories:
flag_continue = True

## Check that slackexport_folder_path exists:
if exists(slackexport_folder_path)==False:
    print('Please enter a valid path to the source directory')
    flag_continue = False
else:
    ## If analysing one channel, check that its directory exists, and default to the 0-th element of channels_names:
    ## channels_names = [ exportname ] for one exportchannel
    ## channels_names = [channel0, channel1, ...] for all the channels
    if analyze_all_flag == False:
        if exists(f"{slackexport_folder_path}/{exportname}")==False:
            print(f"The source directory for the channel '{exportname}' was not found in {slackexport_folder_path}")
            flag_continue = False
        else:
            channels_names = [exportname]
    else:
        all_in_sourceDir = listdir(slackexport_folder_path)
        channels_names = [all_in_sourceDir[i] for i in range(len(all_in_sourceDir)) if isdir(f"{slackexport_folder_path}/{all_in_sourceDir[i]}")==True]

    
    ## Check the names of json files (yyyy-mm-dd.json) and stores them:
    ## all_channels_jsonFiles_dates = [ [exportname_json0, exportname_json1, ...] ] for one exportchannel
    ## all_channels_jsonFiles_dates = [ [channel0_json0, channel0_json1, ...], [channel1_json0, channel1_json1, ...], ... ] for all the channels
    all_channels_jsonFiles_dates = []
    #all_channels_jsonFiles_others = []
    for channel in channels_names:
        channel_jsonFiles_dates, channel_jsonFiles_others = review_format_of_jsonFiles( listdir(f"{slackexport_folder_path}/{channel}") )
        all_channels_jsonFiles_dates.append(channel_jsonFiles_dates)
        #all_channels_jsonFiles_others.append(channel_jsonFiles_others)  
        #   IP20241118: "all_channels_jsonFiles_others.append"  is senseless, because "other" files could have dofferent inner JSON-structure
        
    ## Check that the users.json files exists:
    if exists(f"{slackexport_folder_path}/users.json")==False:
        print('File "users.json" was not found in the source directory')
        flag_continue = False

    #  !!! IP2024118  need to check if exist File "channels.json" 


#print(flag_continue) 
print(channels_names)
#print(all_channels_jsonFiles_others)
#print(all_channels_jsonFiles_dates)

['3d-art-blender-landmarks', '3d-art-team-a-landmarks', '3d-art-team-b-landmarks', 'general', 'JSONs_converted', 'RET_converted', '_converted']


In [68]:
### Definition of funtions used later in the analysis:

# !!! IP20241118  check, why 'n_d'  appears in the "text"s cells in messages

def replace_empty_space(df, column):
    """Function to replace empty spaces "" with the string 'n/a' for a given column"""
    for i in range(len(df)):
        if df.at[i,column] == "":
            df.at[i,column] = 'n_d'  
            
def replace_NaN(df, column):
    """Function to replace missing values with the string 'n/a' for a given column """
    df[column] = df[column].fillna('n-d')


In [69]:

all_channels_df = pd.read_json(f"{slackexport_folder_path}/channels.json")
all_channels_df[0:1]


,id,name,created,creator,is_archived,is_general,members,pins,topic,purpose
0,C020HQB61PF,general,1619815937,U02063W7Z1V,False,True,"[U02063W7Z1V, U0450DR40FP, U04HGHJ291Q, U04JL6...","[{'id': '1714832626.257419', 'type': 'C', 'cre...","{'value': '', 'creator': '', 'last_set': 0}",{'value': 'This is the one channel that will a...


In [70]:

def get_all_channels_info(slackexport_folder_path):
    
    # ! IP20241118   "The primary features of all_users_df are: "  - is correct?  may be - "all_channels_df" ??
    """
    This function exports the file channels.json into the dataframe all_channels_df and filters/format relevant features.
    The primary features of all_users_df are: 
        id, name, created, creator, is_archived, is_general, members, pins, topic, purpose.
    The secondary features of 'pins' are:
        id, type, created, user, owner.
        Generally a list of dictionaries.
    The secondary features of 'topic' are:
        value, creator, last_set.
    """
    ### Export channels.json to dataframe    
    all_channels_df = pd.read_json(f"{slackexport_folder_path}/channels.json")
    

    # ! IP20241118 code below not take in count - which JSONs _supposed-to-present_ in the export folder
    #  code below store only JSONs which physically presented in the time of iterating folder
    #  think, all_channels_df should preserv initial list of JSON's, which stored in "channels.json" originally
    #  to provide manual checking of folder/jsons consistence


    ## Format relevant features on all_channels_df:
    all_json_files = []
    for i in range(len(all_channels_df)):
        ## Writes the list of members into a string separated by commnas:
        tmp_list = all_channels_df.at[i, 'members']
        members_str = "".join(f"{tmp_list[j]}, " for j in range(len(tmp_list)))
        all_channels_df.at[i,'members'] = members_str[:-2]
        ## Add the 'purpose' of the channel:
        all_channels_df.at[i,'purpose'] = all_channels_df.at[i,'purpose']['value']
        ## Adds a list with the channel's json_files with the correct format (yyyy-mm-dd.json):
        channel_path = f"{slackexport_folder_path}/{all_channels_df.at[i,'name']}"
        
        #print("in the  def'get_all_channels_info' channel_path =>> "+channel_path )

        ## Check that the   channel_path    exists:   #IP20241118
        if exists(channel_path)==True:
            list_names_dates, list_names_others = review_format_of_jsonFiles(listdir(channel_path))
            all_json_files.append(list_names_dates)
        else:
            all_json_files.append('n/d')

        
    all_channels_df['json_files'] = all_json_files
    
    ## Keep the relevant features:
    all_channels_df = all_channels_df[['id', 'name', 'created', 'creator', 'is_archived', 'is_general', 'members', 'purpose', 'json_files']]

    ## Handle missing values or empty strings:
    replace_empty_space(all_channels_df, 'members')
    replace_empty_space(all_channels_df, 'purpose')
    
    return all_channels_df


def get_all_users_info(slackexport_folder_path):
    """
    This function exports the file users.json into the dataframe all_users_df and filters/format relevant features.
    The primary features of all_users_df are: 
        id, team_id, name, deleted, color, real_name, tz, tz_label, tz_offset, profile, is_admin, is_owner,
        is_primary_owner, is_restricted,is_ultra_restricted, is_bot, is_app_user, updated, is_email_confirmed,
        who_can_share_contact_card, is_invited_user, is_workflow_bot, is_connector_bot.
    Among the secondary features of 'profile', there are:
        title, phone, skype, real_name, real_name_normalized, display_name, display_name_normalized, fields, 
        status_text, status_emoji, status_emoji_display_info, status_expiration, 
        avatar_hash, image_original, is_custom_image, email, huddle_state, huddle_state_expiration_ts, 
        first_name, last_name, image_24, image_32, image_48, image_72, image_192, image_512, image_1024, 
        status_text_canonical, team.
    """
    ## Read users.json as a dataframe:
    all_users_df = pd.read_json(f"{slackexport_folder_path}/users.json")
    
    ## Keep relevant features on all_users_df:
    for i in range(len(all_users_df)):
        all_users_df.at[i, 'display_name'] = all_users_df.at[i, 'profile']['display_name']
        #all_users_df.at[i, 'profile_title'] = all_users_df.at[i, 'profile']['title']  ## Contain a lot of missing values. Display_name seems more representative.
    all_users_df = all_users_df[['id', 'team_id', 'name', 'deleted', 'display_name', 'is_bot']]#,'profile_title']]
    
    ## Handling missing values in all_users_df:
    replace_empty_space(all_users_df, 'display_name')
    replace_empty_space(all_users_df, 'name')
    replace_empty_space(all_users_df, 'team_id')
    replace_empty_space(all_users_df, 'id')
    
    return all_users_df
    
def slack_json_to_dataframe(slack_json):
    """ Function to extract channel's messages from a JSON file """
    
    messages_df = pd.DataFrame(columns=["msg_id", "ts", "user", "type", "text", 
                                        "reply_count", "reply_users_count", 
                                        "ts_latest_reply", "ts_thread", "parent_user_id"])
    
    # ! IP20241118  "= None"  looks not good.  we need explicit sign of data was not provided
    #     so, replace in the code ::  " = None "  with " = 'n/d' "

    for message in range(len(slack_json)):
        #if 'files' in slack_json[message] and slack_json[message]['files']:            #AG:commented out
        #    messages_df.at[message, "msg_id"] = slack_json[message]['files'][0]['id']  #AG:commented out
        if 'client_msg_id' in slack_json[message]:
            messages_df.at[message, "msg_id"] = slack_json[message]['client_msg_id']
        elif 'subtype' in slack_json[message]:                                       #AG:added
            messages_df.at[message, "msg_id"] = slack_json[message]['subtype']       #AG:added
        else:
            messages_df.at[message, "msg_id"] = None #'n/a'
            
        if 'ts' in slack_json[message]:
            messages_df.at[message, "ts"] = slack_json[message]['ts']
        else:
            messages_df.at[message, "ts"] = None #'n/a'  # 20241110-2
            
        messages_df.at[message, "user"] = slack_json[message].get('user', None) #'n/a')
        
        if 'type' in slack_json[message]:
            messages_df.at[message, "type"] = slack_json[message]['type']
        else:
            messages_df.at[message, "type"] = None #'n/a'  # 20241110-2
        
        if 'text' in slack_json[message]:
            messages_df.at[message, "text"] = slack_json[message]['text']
            #AG:begin
            #txt = slack_json[message]['text']
            #if txt=="" and 'attachments' in slack_json[message]:
                ## Covers cases like in 2023/01/11 with the Slackbot messages:
            #    messages_df.at[message, "text"] = slack_json[message]['attachments'][0]['from_url']
                #messages_df.at[message, "user"] = slack_json[message]['bot_id']   #overwrittes the empty string, particular to this case.
            #else:
            #    messages_df.at[message, "text"] = txt
            #AG:end
        else:
            messages_df.at[message, "text"] = None #'n/a'  # 20241110-2

        if 'reply_count' in slack_json[message]:
            messages_df.at[message, "reply_count"] = slack_json[message]['reply_count']
            messages_df.at[message, "reply_users_count"] = slack_json[message]['reply_users_count']
            messages_df.at[message, "ts_latest_reply"] = slack_json[message]['latest_reply']
        else:
            messages_df.at[message, "reply_count"] = None#'n/a'  # 20241110-2
            messages_df.at[message, "reply_users_count"] = None#'n/a'  # 20241110-2
            messages_df.at[message, "ts_latest_reply"] = None#'n/a'  # 20241110-2

        if 'parent_user_id' in slack_json[message]:
            messages_df.at[message, "ts_thread"] = slack_json[message]['thread_ts']
            messages_df.at[message, "parent_user_id"] = slack_json[message]['parent_user_id']
        else:
            messages_df.at[message, "ts_thread"] = None#'n/a'  # 20241110-2
            messages_df.at[message, "parent_user_id"] = None#'n/a'  # 20241110-2
            
    return messages_df
    

def get_channel_messages_df(export_path, curr_channel_name, json_list):
    """ Extracts all the messages of a given channel from its JSON files, and stores them on a data frame """
    channel_messages_df = pd.DataFrame(columns=["msg_id", "ts", "user", "type", "text",
                                                "reply_count", "reply_users_count",
                                                "ts_latest_reply", "ts_thread", "parent_user_id"])
                                                # ,"channel_folder", "json_name", "json_mod_date"])          #_IP
    
    # Iterate over JSONs inside the current channel's folder:
    for file_day in range(len(json_list)):
        #filejson_path = f"{parentfolder_path}/{channels_json[curr_channel_name]['dayslist'][file_day]}"
        filejson_path = f"{export_path}/{curr_channel_name}/{json_list[file_day]}" #AG
        
        with open(filejson_path, encoding='utf-8') as f:
            import_file_json = load(f)
        import_file_df = slack_json_to_dataframe(import_file_json)
        import_file_df['json_name'] = json_list[file_day]
        import_file_df['json_mod_ts'] = getmtime(filejson_path)
        
        channel_messages_df = pd.concat([channel_messages_df, import_file_df], axis=0, ignore_index=True) 
    
    channel_messages_df['channel_folder'] = curr_channel_name   #IP
    return channel_messages_df


def get_channel_users_df(channel_messages_df, users_df ):
    """Returns a data frame with the information of the users in current channel"""
    # Initialize channel_users_df as a copy of users_df:
    channel_users_df = users_df.copy()
    # Find the unique set of users in channel:
    channel_users_list = channel_messages_df['user'].unique()
    # Collect the indices of the users that are NOT in the channel:
    indices_to_drop = [i for i in range(len(users_df)) if users_df.at[i,'id'] not in channel_users_list ]
    # Drop the rows on indices_to_drop:
    channel_users_df.drop(channel_users_df.index[indices_to_drop], inplace=True)
    return channel_users_df

def add_users_info_to_messages(df_messages, df_users):
    for index in df_messages.index.values:
        i_df = df_users[df_users['id']==df_messages.at[index,'user']]
        if i_df['display_name'].shape[0]==0:        ##AG: 'USLACKBOT' is a special case
            df_messages.at[index, 'name'] =  df_messages.at[index, 'user']
            df_messages.at[index, 'display_name'] =  df_messages.at[index, 'user']
            df_messages.at[index, 'is_bot'] =  True
        else:
            df_messages.at[index, 'name'] = i_df['name'].values
            df_messages.at[index, 'display_name'] = i_df['display_name'].values
            df_messages.at[index, 'is_bot'] = i_df['is_bot'].values
        del i_df
    #del channel_users_df



# !!!  IP20241118   time convert is not proper. initial time in JSONs is NOT "EST" 
#                       may be - it provided in GMT+0
#
# !!! encounter problem/bag :: 
#    "" AmbiguousTimeError: Cannot infer dst time from 2023-11-05 01:21:45.201458931, try using the 'ambiguous' argument "
# need to be fixed in the code

def ts_to_tz(df, original_column_name, new_column_name):
    """Transforms timestamps in a dataframe's column to dates on the "US/Central" timezone"""
    df[original_column_name] = pd.to_numeric(df[original_column_name], errors='coerce')   #_IP
    tzs = []
    for i in range(len(df)):
        if df.at[i,original_column_name] == None:
            i_date = '0000-00-00 00:00:00'
        else:
            # Convert the time to CST (UTC-6)
            #df['datetime_cst'] = df['datetime_gmt'].dt.tz_convert('America/Chicago')

            # IP20241119
            i_date = pd.to_datetime(df.at[i,original_column_name], unit='s').tz_localize('UTC').tz_convert('US/Central')

            #i_date = pd.to_datetime(df.at[i,original_column_name], unit='s').tz_localize('US/Eastern').tz_convert('US/Central')
            i_date = datetime.strftime(i_date,"%Y-%m-%d %H:%M:%S")
        tzs.append(i_date)
    df[[original_column_name]].astype('datetime64[s]')
    df[original_column_name] = tzs
    df.rename(columns={original_column_name: new_column_name}, inplace=True)



# IP20241118   should extract not only 1st url from "text", but all of url's
def extract_urls(df):
    """"""
    extractor = URLExtract()
    for i in range(len(df)):
        text = df.at[i,'text']
        if 'https' in text:
            url = extractor.find_urls(text)
            df.at[i,'URL'] = url
        else:
            df.at[i,'URL'] = '' # None   IP2024118


def user_id_to_name(df_messages, df_users):
    """"""
    for i in range(len(df_messages)):
        text = df_messages.at[i,'text']
        matches = re.findall(r'<+@[A-Za-z0-9]+>',text)
        if len(matches)>0:
            for match in matches:
                user = match[2:-1]
                name = df_users[df_users['id']==user]['display_name'].values[0]
                if name!='n/a':
                    text = re.sub(f"<@{user}>", f"{name}", text)
            df_messages.at[i,'text'] = text
    return df_messages


def apply_excel_adjustments(file_path, curr_channel_name):
    """ Excel file formatting/adjustments with  openpyxl (IP) """
    wb = load_workbook(file_path)
    ws = wb.active
    # Set the column width
    column_widths = {
        'B': 19, 'C': 15, 'E': 25, 'K': 25, 'L': 19, 'M': 19, 'N': 13, 'O': 13     
    }
    # Apply the column widths
    for col, width in column_widths.items():
        ws.column_dimensions[col].width = width
    #
    # Freeze the first row (Row 1)
    ws.freeze_panes = 'A2'
    # Set font size and bold for the first row
    font = Font(size=9, bold=True)
    # Define the RGB color
    fill = PatternFill(start_color="e7c9fb", end_color="e7c9fb", fill_type="solid")
    # Apply the color to the first row (row 1)
    # Apply the font formatting to the first row (Header row)
    for cell in ws[1]:
        cell.font = font
        cell.fill = fill
    #
    # Rename the sheet
    ws_title = curr_channel_name 
    ws_title = ws_title[:31]
    ws.title = ws_title 
    #
    # Save the changes to the Excel file
    wb.save(file_path)

In [71]:
### Main analysis:
if flag_continue==False:
    print("Please review the input information")
else:    
    print(datetime.now().time(), 'Started analysis after sanity checks' )
    print("slackexport_folder_path =>> "+slackexport_folder_path)
    ### Extract the channels and users information into dataframes:
    channels_df = get_all_channels_info(slackexport_folder_path)
    print(datetime.now().time(), 'Obtained channels_df')
    users_df = get_all_users_info(slackexport_folder_path)
    print(datetime.now().time(), 'Obtained users_df')
    
    ### Write all channel's info to .xlsx files, if requested by user:
    if write_all_channels_info==True:
        slack_export_channel_filename = "_all_channels"
        slack_export_channel_folder_path_xlsx = f"{exporting_directory}/{slack_export_channel_filename}{'.xlsx'}"
        channels_df.to_excel(slack_export_channel_folder_path_xlsx, index=False)
        print(datetime.now().time(), 'Wrote channels_df to xlsx file')  
    
    ### Write all users's info to .xlsx files, if requested by user:
    if write_all_users_info==True:
        slack_export_user_filename = "_all_users"        
        slack_export_user_folder_path_xlsx = f"{exporting_directory}/{slack_export_user_filename}{'.xlsx'}" #_IP
        users_df.to_excel(slack_export_user_folder_path_xlsx, index=False) #_IP
        print(datetime.now().time(), 'Wrote users_df to xlsx file')

    ### Iterate over channel's folders:
    print(datetime.now().time(), 'Starting loop over channels', '\n')
    for i_channel in range(len(channels_names)):

        ## Define the name of the current channel and the source path containing its json files:
        curr_channel_name = channels_names[i_channel] 
        parentfolder_path = f"{slackexport_folder_path}/{curr_channel_name}" 
        print(curr_channel_name, datetime.now().time(), ' Set-up channel name and path to directory')
        
        ## Collect all the current_channel's messages in channel_messages_df through the function get_channel_messages_df:
        json_list = all_channels_jsonFiles_dates[i_channel]
        channel_messages_df = get_channel_messages_df(slackexport_folder_path, curr_channel_name, json_list)  
        print(curr_channel_name, datetime.now().time(), ' Collected channel messages from the json files')

        ## Collect all the users in the current channel through the function get_channel_users_df:
        channel_users_df = get_channel_users_df(channel_messages_df, users_df )
        print(curr_channel_name, datetime.now().time(), ' Collected users in current channel')
        
        ## Use channel_users_df to fill-in the user's information in channel_messages_df:  (define in function onces tested)
        add_users_info_to_messages(channel_messages_df, channel_users_df)
        print(curr_channel_name, datetime.now().time(), ' Included the users information on channel_messages_df')
        
        ## Replace user and team identifiers with their display_names whenever present in a message:
        user_id_to_name(channel_messages_df, users_df)  #(debugging!)
        print(curr_channel_name, datetime.now().time(), " User's id replaced by their names in messages")

        ## Extract hyperlinks from messages, if present (extracted as a list; edit if needed):
        extract_urls(channel_messages_df)
        print(curr_channel_name, datetime.now().time(), ' URLs extracted from messages')

        ## Change format of the time in seconds to a date in the CST time-zone: (Pending 'ts_latest_reply' and 'ts_thread'!)
        channel_messages_mindate = pd.to_datetime(np.float64(channel_messages_df['ts']), unit='s').min().date()
        channel_messages_maxdate = pd.to_datetime(np.float64(channel_messages_df['ts']), unit='s').max().date()
        ts_to_tz(channel_messages_df, 'ts', 'msg_date')
        ts_to_tz(channel_messages_df, 'json_mod_ts', 'json_mod_date')
        print('main_analysys ->>',curr_channel_name, datetime.now().time(), ' Formated the dates and times in the dataframe')
            
        ## Reorder the columns in channel_messages_df, if necessary:
        #channel_messages_df = channel_messages_df[['channel', 'json_name', 'json_mod_date', 'user', 'name', 'display_name', 'ts', 'msg_id', 'type', 'text']]
        #channel_messages_df.index = ['']*len(channel_messages_df)
        
        ## Write channel_messages_df to a .xlsx file:
        channel_messages_filename = f"{curr_channel_name}_{channel_messages_mindate}_to_{channel_messages_maxdate}"
        channel_messages_folder_path = f"{exporting_directory}/{channel_messages_filename}.xlsx"
        channel_messages_df.to_excel(f"{channel_messages_folder_path}", index=False)
        apply_excel_adjustments(f"{channel_messages_folder_path}",curr_channel_name)  #AG: defined this routine in the function apply_excel_adjustments
        print(curr_channel_name, datetime.now().time(), ' Wrote curated messages to xlsx files', '\n')

print(datetime.now().time(), 'Done')

14:32:24.391721 Started analysis after sanity checks
slackexport_folder_path =>> E:\_RET_slack_export\RebeccaEverlene Slack export Apr 30 2021 - Oct 3 2024-2short
14:32:24.457562 Obtained channels_df
14:32:24.824139 Obtained users_df
14:32:24.911788 Wrote channels_df to xlsx file
14:32:25.273084 Wrote users_df to xlsx file
14:32:25.273084 Starting loop over channels 

3d-art-blender-landmarks 14:32:25.273084  Set-up channel name and path to directory
3d-art-blender-landmarks 14:32:27.757159  Collected channel messages from the json files
3d-art-blender-landmarks 14:32:27.790370  Collected users in current channel
3d-art-blender-landmarks 14:32:29.290166  Included the users information on channel_messages_df
3d-art-blender-landmarks 14:32:29.855785  User's id replaced by their names in messages
3d-art-blender-landmarks 14:32:30.513277  URLs extracted from messages
main_analysys ->> 3d-art-blender-landmarks 14:32:31.669408  Formated the dates and times in the dataframe
3d-art-blender-lan

KeyError: 'json_mod_ts'

In [ ]:
channel_messages_df.head()